<a href="https://colab.research.google.com/github/rachelwijaya/MovieRecommendation/blob/main/MovieRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [167]:
# import csv files
import time
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies (1).csv
Saving user_ratings.csv to user_ratings (1).csv


In [168]:
import io
movies_df = pd.read_csv(io.BytesIO(uploaded['movies.csv']))
user_ratings_df = pd.read_csv(io.BytesIO(uploaded['user_ratings.csv']))
# Dataset is now stored in a Pandas Dataframe

# Jaccard Similarity

In [181]:
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

# Returns a condensed matrix of all the distances

 # Crosstabulation
movies_genre_df = pd.crosstab(movies_df['title'], movies_df['genres'])

# Find distance between all items
# WARNING: please be patient, calculating this is going to take a minute or two
t1 = time.perf_counter()
jaccard_distances = pdist(movies_genre_df.values, metric='jaccard')

t2 = time.perf_counter()
print('time elapsed: ', t2-t1)
# print(jaccard_distances)

# Get the data into a rectangular shape based on the previous matrix
square_jaccard_distances = squareform(jaccard_distances)
# print(square_jaccard_distances)

time elapsed:  137.99027016400214


In [182]:
# Convert the distances to a square matrix
jaccard_similarity_array = 1 - square_jaccard_distances
# print(jaccard_similarity_array)

# Create a dataframe for the distance of each book according to its genre
distance_df = pd.DataFrame(jaccard_similarity_array,
                           index=movies_genre_df.index,
                           columns=movies_genre_df.index)
# distance_df.head()

In [183]:
# Find a similar movie
movie_name = input("Enter a movie title: ")

# Find the values for the movie
# movie_name_genres = movies_genre_df[movies_genre_df.index == movie_name]
jaccard_similarity_series = distance_df.loc[movie_name]

# Sort these values from highest to lowest
ordered_similarities = jaccard_similarity_series.sort_values(ascending = False)

# Print the results
print("\nMovies you would like:")
print(ordered_similarities.head())


Enter a movie title: Jumanji (1995)

Movies you would like:
title
Escape to Witch Mountain (1975)              1.0
Golden Compass, The (2007)                   1.0
Indian in the Cupboard, The (1995)           1.0
The Cave of the Golden Rose (1991)           1.0
Darby O'Gill and the Little People (1959)    1.0
Name: Jumanji (1995), dtype: float64


# Cosine Similarity

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#helper functions
cv = CountVectorizer()

def get_title_from_index(index):
	return movies_df[movies_df.index == index]["title"].values[0]

def get_index_from_title(title):
	return movies_df[movies_df.title == title]["index"].values[0]

In [ ]:
#selecting the feature
features = ['title', 'genres']
for feature in features:
	movies_df[feature] = movies_df[feature].fillna('')

In [ ]:
#combining all selected features into one column
def combined_features(row):
    try:
        return row['title'] + " " + row['genres']
    except:
        print("Error:", row)

movies_df["index"] = range(len(movies_df))
movies_df["combined_features"] = movies_df.apply(combined_features,axis=1)
                                                    #will pass each row individually instead of columns

print("combined features:", movies_df["combined_features"].head())

combined features: 0    Toy Story (1995) Adventure|Animation|Children|...
1            Jumanji (1995) Adventure|Children|Fantasy
2               Grumpier Old Men (1995) Comedy|Romance
3        Waiting to Exhale (1995) Comedy|Drama|Romance
4            Father of the Bride Part II (1995) Comedy
Name: combined_features, dtype: object


In [ ]:
def recommendedMovieList(user_movie):
    #creating a count matrix from the combined column
    count_matrix = cv.fit_transform(movies_df["combined_features"])

    #finding the cosine similarity value from the count matrix
    cosine_sim = cosine_similarity(count_matrix) 

    #getting the movieId from its title
    movie_index = get_index_from_title(user_movie)

    #getting a list of similar movies by calculating their similarity scores
    similar_movies = list(enumerate(cosine_sim[movie_index]))

    #sorting the similarity scores in descending order
    return sorted(similar_movies, key=lambda x:x[1], reverse=True)

In [ ]:
#asking for user input
user_movie = input("Please enter a movie: ")

Please enter a movie: Jumanji (1995)


In [ ]:
t1 = time.perf_counter()
movie_list = recommendedMovieList(user_movie)

#printing the titles of 5 movies that are found to be similar
i = 0
for movie in movie_list:
		print(get_title_from_index(movie[0]))
		i += 1
		if i > 5:
			break
t2 = time.perf_counter()
print('time_elapsed: ', t2-t1)

Jumanji (1995)
Tall Tale (1995)
Casper (1995)
Toy Story (1995)
Balto (1995)
Gordy (1995)
time_elapsed:  1.0777893090007638


# User-based Recommendation (cosine_similarity)
cosine similarity is the measure of the angle between two documents in the high dimensional matrix space

as seen in this two dimensional example [here](https://imgur.com/a/NH9FMlH)

all values are between 0 and 1 where 1 is an exact match

In [185]:
ratings = pd.merge(movies_df,user_ratings_df).drop(['genres', 'timestamp'], axis=1)
ratings.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [186]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()

#Remove movies that have less than 10 user's rating (that are not NaN) and fill NaN values with 0
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)

In [187]:
#similarity Matrix
item_similarity_df = userRatings.corr(method='pearson') #in-build method of dataframe of standardizing rating given by all user 
item_similarity_df.head(100)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.063117,-0.023768,0.143482,0.011998,0.087931,0.224052,0.034223,0.009277,0.008331,0.049700,0.176071,0.131993,0.104500,0.047271,0.184452,0.092824,0.084066,0.166599,0.205977,0.187019,0.006377,-0.027152,0.035609,-0.014293,-0.027713,0.099271,-0.028021,0.107001,0.117119,-0.028967,0.144642,0.109059,0.021677,0.131405,0.115386,0.124265,-0.022021,0.255706,0.024466,...,-0.001830,0.006420,-0.019439,0.194225,0.080328,0.160591,0.061280,-0.027261,0.031088,0.043787,0.081831,0.075004,0.213522,0.195512,0.087015,0.007059,-0.011004,0.049537,0.119975,0.098020,0.115369,0.016625,0.042313,-0.023309,0.021379,0.172502,0.201374,0.248535,0.088114,0.257195,0.017477,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,0.187953,0.062174,0.353194
(500) Days of Summer (2009),0.063117,1.000000,0.142471,0.273989,0.193960,0.148903,0.142141,0.159756,0.135486,0.200135,0.297152,0.072835,0.226574,0.087240,0.342948,0.038255,0.040104,0.241906,0.001739,0.113616,0.030603,0.274272,0.285043,0.094029,0.284552,0.194200,0.086447,0.205511,0.048670,0.149167,0.157145,0.249836,0.343028,-0.034688,0.252980,0.174618,0.344805,0.023625,0.156628,0.314037,...,0.241685,0.207393,0.040382,0.100958,0.225268,0.044576,0.031382,0.265563,0.303689,0.126481,0.388055,0.023020,0.031399,0.184228,0.320018,0.238263,0.288610,0.349784,0.274941,0.190061,0.034780,0.407336,0.029789,0.144834,-0.012948,0.140981,0.066077,0.073476,0.053263,0.053021,0.374515,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.053614,0.241092,0.125905
10 Cloverfield Lane (2016),-0.023768,0.142471,1.000000,-0.005799,0.112396,0.006139,-0.016835,0.031704,-0.024275,0.272943,-0.027835,-0.029983,0.181245,0.017315,-0.019197,0.049227,-0.020711,0.097364,0.012478,0.099310,-0.025037,0.205739,0.082904,-0.030166,0.309963,0.292432,0.092291,-0.026642,-0.008488,0.228241,0.231702,0.263072,0.170440,-0.019994,0.104868,0.081960,0.145923,0.033310,0.147068,0.025681,...,0

Another method of standardizing (instead of using the in-build pearson correlation function)

```
def standardize(row):
  new_row = (row - row.mean())/(row.max() - row.min())
  return new_row

ratings_std = ratings.apply(standardize)
print(ratings_std)

item_similarity = cosine_similarity(ratings_std.T)
print(item_similarity)
```



What is the purpose of standardizing?

To correct any user rating that are to harsh or to lenient 
for example,

We have a user's rating values of [1,5,0]

```
def standardize(row):

  #mean = (1+5+0)/3 = 2
  new_row = (row - row.mean())/(row.max() - row.min())

  #row - row.mean() = [(1,-2), (5,-2), (0,-2)] = [-1,3,-2]
  #new meane = 0

  #row.max() - row.min() = 5
  #new_row = [-1/5, 3/5 , -2/5]
  #new range = 3/5 - (-2/5) = 1

  return new_row
```



In [188]:
#Find the similarity between movies
def get_similar(movie_name,rating):
    similar_ratings = item_similarity_df[movie_name]*(rating-2.5)
    #Scale it based on the rating the user gave
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [191]:
#Make a recommendation
#sample input for Cally to insert = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
print("NOTE: input format, please enter movie name then the rating(out of 5)")
movie_lst = [ ] 
n = int(input("Enter number of movies you want to rate : ")) 
  
for i in range(0, n): 
    ele = [input(), int(input())] 
    movie_lst.append(ele) 

similar_movies = pd.DataFrame()
for movie,rating in movie_lst:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
print("Here are a list of 20 movies reccomended for you \n")
print(similar_movies.sum().sort_values(ascending=False).head(20))

NOTE: input format, please enter movie name then the rating(out of 5)
Enter number of movies you want to rate : 1
Jumanji (1995)
6
Here are a list of 20 movies reccomended for you 

Jumanji (1995)                                               3.500000
Lion King, The (1994)                                        1.683983
Mrs. Doubtfire (1993)                                        1.551120
Mask, The (1994)                                             1.517662
Nightmare Before Christmas, The (1993)                       1.506065
Home Alone (1990)                                            1.495562
Casper (1995)                                                1.479201
Santa Clause, The (1994)                                     1.452956
Ace Ventura: When Nature Calls (1995)                        1.444827
Jurassic Park (1993)                                         1.391848
Beauty and the Beast (1991)                                  1.359245
Aladdin (1992)                                  

# User-based Recommendation (K-Nearest Neighbours)

In [184]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix # convert into array matrix
from sklearn.neighbors import NearestNeighbors # unsupervised learning

# create dataframes from csv files
movies_df = pd.read_csv('movies.csv', usecols=['movieId','title'], dtype={'movieId': 'int32', 'title': 'str'})
rating_df = pd.read_csv('user_ratings.csv', usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

# print("Movies Dataframe:\n", movies_df.head(10))
# print("User Rating Dataframe:\n", rating_df.head(10))

# Merge two of the dataframes on movieId as this is a common feature
knnDf = pd.merge(rating_df, movies_df, on='movieId')

movie_rated_df = knnDf.dropna(axis=0, subset=['title'])
# select title as the set that we want
# count the amount of ratings done of each movie, reset index so that title is not the index
# This ensures that if let's say a movie has only 1 user who rates it a 5 stars, doesn't skew the data
movie_rated_count = (movie_rated_df.groupby(by=['title'])["rating"]).count().reset_index().rename(columns={'rating':'ratingCount'})[['title', 'ratingCount']]


rateTotalMovieCount = movie_rated_df.merge(movie_rated_count, left_on = 'title', right_on = 'title', how = 'left')

# Create pivot matrix
threshold = 30
rate_popular_movie = rateTotalMovieCount.query('ratingCount >= @threshold')
movie_features_pivot = rate_popular_movie.pivot_table(index ='title', columns='userId', values='rating').fillna(0)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
movie_features_df_matrix = csr_matrix(movie_features_pivot.values)
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine', metric_params=None, n_jobs=None, n_neighbors=4, p=1.5, radius=1.0)

user_movie = input("Please enter a movie: ")
index = movie_features_pivot.index.get_loc(user_movie)
distances, indices = model_knn.kneighbors(movie_features_pivot.iloc[index, :].values.reshape(1, -1), n_neighbors = 6)


for i in range(0, len(distances.flatten())):
    if i == 0:
        print("Recommendations for {0}:\n".format(movie_features_pivot.index[index]))
    else:
        print("{0}: {1}, with distance of {2}:".format(i, movie_features_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Please enter a movie: Jumanji (1995)
Recommendations for Jumanji (1995):

1: Lion King, The (1994), with distance of 0.411562442779541:
2: Mrs. Doubtfire (1993), with distance of 0.45018184185028076:
3: Mask, The (1994), with distance of 0.45501887798309326:
4: Jurassic Park (1993), with distance of 0.4619544744491577:
5: Home Alone (1990), with distance of 0.47512364387512207:
